# How to debug your LLM apps

Like building any type of software, at some point you'll need to debug when building with LLMs. A model call will fail, or model output will be misformatted, or there will be some nested model calls and it won't be clear where along the way an incorrect output was created.

There are three main methods for debugging:

- Verbose Mode: This adds print statements for "important" events in your chain.
- Debug Mode: This add logging statements for ALL events in your chain.
- LangSmith Tracing: This logs events to [LangSmith](https://docs.smith.langchain.com/) to allow for visualization there.

|                        | Verbose Mode | Debug Mode | LangSmith Tracing |
|------------------------|--------------|------------|-------------------|
| Free                   | ✅            | ✅          | ✅                 |
| UI                     | ❌            | ❌          | ✅                 |
| Persisted              | ❌            | ❌          | ✅                 |
| See all events         | ❌            | ✅          | ✅                 |
| See "important" events | ✅            | ❌          | ✅                 |
| Runs Locally           | ✅            | ✅          | ❌                 |


## Tracing

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```

Let's suppose we have an agent, and want to visualize the actions it takes and tool outputs it receives. Without any debugging, here's what we see:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>
```

In [5]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

In [6]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

tools = [TavilySearchResults(max_results=1)]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who directed the 2023 film Oppenheimer and what is their age in days?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiti

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'It seems there was an issue accessing the search tool to retrieve the information. However, I can provide the information based on existing knowledge.\n\nThe 2023 film "Oppenheimer" was directed by Christopher Nolan. To calculate his age in days, I need to know his birthdate. Christopher Nolan was born on July 30, 1970. Let\'s calculate his age in days from his birthdate to today.\n\nFirst, we find the total number of days from July 30, 1970, to today\'s date.\n\n1. Calculate the number of days from July 30, 1970, to December 31, 1970.\n2. Add the total number of days for each year from 1971 to 2022.\n3. Add the days from January 1, 2023, to today\'s date.\n\nLet\'s do the calculation:\n\n- From July 30, 1970, to December 31, 1970: 155 days (including the start day)\n- Total days from 1971 to 2022: \\(52 \\times 365\\) days + 13 leap days (1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004,

We don't get much output, but since we set up LangSmith we can easily see what happened under the hood:

https://smith.langchain.com/public/a89ff88f-9ddc-4757-a395-3a1b365655bf/r

## `set_debug` and `set_verbose`

If you're prototyping in Jupyter Notebooks or running Python scripts, it can be helpful to print out the intermediate steps of a chain run.

There are a number of ways to enable printing at varying degrees of verbosity.

Note: These still work even with LangSmith enabled, so you can have both turned on and running at the same time


### `set_verbose(True)`

Setting the `verbose` flag will print out inputs and outputs in a slightly more readable format and will skip logging certain raw outputs (like the token usage stats for an LLM call) so that you can focus on application logic.

In [7]:
from langchain.globals import set_verbose

set_verbose(True)
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who directed the 2023 film Oppenheimer and what is their age in days?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiti

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'I am currently unable to retrieve the information due to a technical issue. However, I can provide the information based on existing knowledge.\n\nThe 2023 film "Oppenheimer" was directed by Christopher Nolan. Christopher Nolan was born on July 30, 1970. To calculate his age in days:\n\n1. First, determine the current date. Let\'s assume today\'s date is December 1, 2023.\n2. Calculate the number of days from his birthdate to today.\n\nHere\'s how you can calculate it:\n- From July 30, 1970, to July 30, 2023, is 53 years.\n- From July 30, 2023, to December 1, 2023, is 124 days.\n\nSince there are 365 days in a non-leap year and considering leap years (approximately 13 extra days accounting for leap years between 1970 and 2023), the calculation would be:\n\\[ 53 \\times 365 + 13 + 124 = 19,372 \\text{ days} \\]\n\nTherefore, Christopher Nolan is approximately 19,372 days old as of December 1, 

### `set_debug(True)`

Setting the global `debug` flag will cause all LangChain components with callback support (chains, models, agents, tools, retrievers) to print the inputs they receive and outputs they generate. This is the most verbose setting and will fully log raw inputs and outputs.

In [8]:
from langchain.globals import set_debug

set_debug(True)
set_verbose(False)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who directed the 2023 film Oppenheimer and what is their age in days?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiti

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'It seems there was an issue accessing the search tool to retrieve the information. However, I can provide the information based on existing knowledge.\n\nThe 2023 film "Oppenheimer" was directed by Christopher Nolan. To calculate his age in days, we need his birthdate, which is July 30, 1970.\n\nTo find his age in days:\n1. Calculate the number of days from his birthdate to today\'s date.\n2. Today\'s date is December 7, 2023.\n\nLet\'s calculate:\n\nChristopher Nolan was born on July 30, 1970. From July 30, 1970, to July 30, 2023, is 53 years. Since 2023 is not a leap year, we calculate the days from July 30, 2023, to December 7, 2023.\n\n- July 30, 2023, to December 7, 2023, is 130 days.\n- 53 years in days (considering 13 leap years from 1972 to 2020) = \\(53 \\times 365 + 13 = 19348\\) days.\n\nAdding these together gives:\n\\[ 19348 + 130 = 19478 \\text{ days} \\]\n\nTherefore, as of Dec